---
# Install the required packages

If needed install the following packages:

In [ ]:
#!pip install datasets transformers imbalanced-learn evaluate

---
# Imports

In [51]:
from transformers import Trainer
from datasets import Dataset, load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from transformers import DistilBertTokenizer
from transformers import DataCollatorWithPadding
from transformers import DistilBertForSequenceClassification
import evaluate
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import make_pipeline
import torch
from transformers import DistilBertForSequenceClassification
# Write your code here. Add as many boxes as you need.

---
# Laboratory Exercise - Run Mode (8 points)

## Introduction

This laboratory assignment's primary objective is to fine-tune a pre-trained language model for detection of toxic sentences (binary classification). 

The dataset contains two attributes: 
- `text`: The sentence which needs to be classified in to toxic/non-toxic
- `label`: 0/1 indicator if the given sentence is toxic

**Note: You are required to perform this laboratory assignment on your local machine.**

# Read the data

The dataset reading is given. Just run the following 2 cells.

**DO NOT MODIFY IT! Just analyse how the data reading was performed, as in the future this part won't be given.**

In [85]:
dataset = load_dataset(
    'csv', 
    data_files={'train': 'dataset_20_12/train.tsv', 'val': 'dataset_20_12/val.tsv','test': 'dataset_20_12/test.tsv'},
    delimiter='\t'
)

**The prediction target column MUST be named 'label' in the dataset !**

See the dataset structure:

In [87]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 3130
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3132
    })
})

---
# Natural Language Processing

## Generate the Tokenizer and Data Collator

For the purposes of this lab you will be using `DistilBertTokenizer` and `DataCollatorWithPadding`.

In [89]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [91]:
collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Tokenize the dataset

For the purposes of lowering the amount of computing set the `max_length` parameter to 15.

In [99]:
dataset = dataset.map(lambda x: tokenizer(x['text'], truncation=True, max_length=15, padding=True, patched=True))

Map:   0%|                                      | 0/1000 [00:00<?, ? examples/s]Keyword arguments {'patched': True} not recognized.
Keyword arguments {'patched': True} not recognized.
Keyword arguments {'patched': True} not recognized.
Keyword arguments {'patched': True} not recognized.
Keyword arguments {'patched': True} not recognized.
Keyword arguments {'patched': True} not recognized.
Keyword arguments {'patched': True} not recognized.
Keyword arguments {'patched': True} not recognized.
Keyword arguments {'patched': True} not recognized.
Keyword arguments {'patched': True} not recognized.
Keyword arguments {'patched': True} not recognized.
Keyword arguments {'patched': True} not recognized.
Keyword arguments {'patched': True} not recognized.
Keyword arguments {'patched': True} not recognized.
Keyword arguments {'patched': True} not recognized.
Keyword arguments {'patched': True} not recognized.
Keyword arguments {'patched': True} not recognized.
Keyword arguments {'patched': True} 

## Define the model

The required model for this lab is the `DistilBertForSequenceClassification`.

In [101]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Define the training arguments

For lowering the compute time I recommend using the following parameters:
- per_device_train_batch_size=128
- per_device_eval_batch_size=128
- **num_train_epochs=1**

In [103]:
from transformers import TrainingArguments

training_args = TrainingArguments(

    output_dir = 'trainer',
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    metric_for_best_model='f1',
    num_train_epochs=1
)

## Load the metrics

Load the best metric for the this specific problem.

In [105]:
# Write your code here. Add as many boxes as you need.
metric = evaluate.load('f1')

### Define the function to compute the metrics

In [107]:
# Write your code here. Add as many boxes as you need.
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

## Generate the Trainer object

In [109]:
# Write your code here. Add as many boxes as you need.

trainer = Trainer(
    model,
    training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['val'],
    compute_metrics=compute_metrics,
    data_collator=collator
)

## Train the model

Use the trainer to train the model.

In [111]:
# Write your code here. Add as many boxes as you need.
trainer.train()

Step,Training Loss


TrainOutput(global_step=8, training_loss=0.6371815204620361, metrics={'train_runtime': 8.3292, 'train_samples_per_second': 120.059, 'train_steps_per_second': 0.96, 'total_flos': 3880880820000.0, 'train_loss': 0.6371815204620361, 'epoch': 1.0})

---
# Evaluate the model

## Generate predictions for the test set

In [77]:
import numpy as np

In [117]:
# Write your code here. Add as many boxes as you need.

predictions = trainer.predict(dataset['test'])

## Extract the predictions (class 0 or 1) from the logits

In [119]:
# Write your code here. Add as many boxes as you need.
preds = np.argmax(predictions.predictions, axis=-1)

## Analyze the performance of the model

In [121]:
from sklearn.metrics import classification_report

# Write your code here. Add as many boxes as you need.
print(classification_report(dataset['test']['label'], preds))

              precision    recall  f1-score   support

           0       0.89      0.43      0.58      1566
           1       0.62      0.95      0.75      1566

    accuracy                           0.69      3132
   macro avg       0.76      0.69      0.67      3132
weighted avg       0.76      0.69      0.67      3132



# Laboratory Exercise - Bonus Task (+ 2 points)

Implement a simple machine learning pipeline to classify if a given text is **toxic** or not. Use TF-IDF vectorization to convert text into numerical features and train a `MultinomialNB` model. If needed use `RandomUnderSampler()`. Compare the results with the transformer model.

In [123]:
# Write your code here. Add as many boxes as you need.
pipeline = make_pipeline(
    
    TfidfVectorizer(),
    MultinomialNB(),
    RandomUnderSampler()
)

---

In [125]:
pipeline.fit(dataset['train']['text'], dataset['train']['label'])

TypeError: All intermediate steps of the chain should be estimators that implement fit and transform or fit_resample (but not both) or be a string 'passthrough' 'MultinomialNB()' (type <class 'sklearn.naive_bayes.MultinomialNB'>) doesn't)

In [127]:
preds_nb = pipeline.predict(dataset['test']['text'])

AttributeError: This 'Pipeline' has no attribute 'predict'

In [29]:
print(classification_report(dataset['test']['label'], preds_nb))

NameError: name 'preds_nb' is not defined